In [ ]:
# Autorzy: Michał Lompart, Jan Kocierz, Karol Strzępek
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML # Do wyświetlania animacji w notatniku
plt.rcParams['animation.embed_limit'] = 100.0

# Definicja układu równań różniczkowych dla zmodyfikowanego modelu drapieżnik-ofiara
def modified_lotka_volterra(M_current, t, a, K, b, f, c):
    """
    Funkcja definiująca układ równań różniczkowych Lotki-Volterry
    z logistycznym wzrostem populacji ofiar.

    Parametry:
    M_current (array): Tablica zawierająca liczebność populacji [X, Y] w danym czasie.
                       X - liczebność populacji ofiar
                       Y - liczebność populacji drapieżników
    t (float): Czas (zmienna niezależna).
    r=a (float): Wewnętrzny współczynnik wzrostu populacji ofiar.
    K (float): Pojemność środowiska dla ofiar.
    b (float): Współczynnik drapieżnictwa (skuteczności ataku).
    f (float): Współczynnik konwersji biomasy ofiar na wzrost drapieżników.
    c (float): Wewnętrzny współczynnik śmiertelności drapieżników.

    Zwraca:
    list: Lista pochodnych [dXdt, dYdt] w danym czasie.
    """
    X, Y = M_current
    dXdt = a * X * (1 - X / K) - b * X * Y
    dYdt = f * X * Y - c * Y
    return [dXdt, dYdt]

# Wspólny wektor czasu dla symulacji
t_vector = np.linspace(0, 300, 300) # Czas od 0 do 100 jednostek, 500 klatek dla płynniejszej animacji

In [ ]:
# --- Scenariusz 1: Wyginięcie Drapieżników (P1 stabilny) ---
print("--- Scenariusz 1: Wyginięcie Drapieżników ---")
# Parametry dla scenariusza 1: fK <= c
a1, K1, b1, f1, c1 = 0.5, 100, 0.01, 0.005, 0.6
print(f"Parametry: a={a1}, K={K1}, b={b1}, f={f1}, c={c1}")
print(f"Sprawdzenie warunku bK <= c: {f1 * K1} <= {c1} -> {f1 * K1 <= c1}")

# Warunki początkowe dla scenariusza 1
X0_1, Y0_1 = 80, 50
M0_1 = [X0_1, Y0_1]

# Symulacja dla scenariusza 1
solution1_M = odeint(modified_lotka_volterra, M0_1, t_vector, args=(a1, K1, b1, f1, c1))
X_sol1, Y_sol1 = solution1_M[:, 0], solution1_M[:, 1]

# Animacja dla scenariusza 1
fig1, (ax1_time, ax1_phase) = plt.subplots(1, 2, figsize=(14, 6))

# Wykres czasowy
ax1_time.set_xlim(0, t_vector.max())
ax1_time.set_ylim(0, max(X_sol1.max(), Y_sol1.max(), K1) * 1.1)
ax1_time.set_xlabel('Czas')
ax1_time.set_ylabel('Liczebność Populacji')
ax1_time.set_title('Scenariusz 1: Wykres Czasowy')
ax1_time.grid(True)
line1_X_time, = ax1_time.plot([], [], lw=2, label='Ofiary (X)', color='blue')
line1_Y_time, = ax1_time.plot([], [], lw=2, label='Drapieżniki (Y)', color='red')
ax1_time.legend()

# Portret fazowy
ax1_phase.set_xlim(0, K1 * 1.1)
ax1_phase.set_ylim(0, max(Y_sol1.max(), Y0_1) * 1.1 if Y_sol1.max() > 0 else 10) # Unikanie ylim(0,0)
ax1_phase.set_xlabel('Liczebność Ofiar (X)')
ax1_phase.set_ylabel('Liczebność Drapieżników (Y)')
ax1_phase.set_title('Scenariusz 1: Portret Fazowy')
ax1_phase.grid(True)
line1_phase, = ax1_phase.plot([], [], lw=2, color='purple')
point1_phase, = ax1_phase.plot([], [], 'o', color='purple', markersize=5)
ax1_phase.plot(K1, 0, 'go', markersize=8, label='Punkt równowagi P1 (K, 0)')
ax1_phase.legend()

plt.tight_layout()

def init1():
    line1_X_time.set_data([], [])
    line1_Y_time.set_data([], [])
    line1_phase.set_data([], [])
    point1_phase.set_data([], [])
    return line1_X_time, line1_Y_time, line1_phase, point1_phase

def animate1(i):
    line1_X_time.set_data(t_vector[:i], X_sol1[:i])
    line1_Y_time.set_data(t_vector[:i], Y_sol1[:i])
    line1_phase.set_data(X_sol1[:i], Y_sol1[:i])
    if i > 0:
      point1_phase.set_data(X_sol1[i-1:i], Y_sol1[i-1:i])
    else:
      point1_phase.set_data([],[])
    return line1_X_time, line1_Y_time, line1_phase, point1_phase

anim1 = FuncAnimation(fig1, animate1, init_func=init1, frames=len(t_vector), interval=10, blit=True)
plt.close(fig1) # Zapobiega wyświetlaniu statycznego wykresu przed animacją
display(HTML(anim1.to_jshtml()))

In [ ]:
# --- Scenariusz 2: Stabilna Koegzystencja (P2 jako stabilny węzeł) ---
print("\n--- Scenariusz 2: Stabilna Koegzystencja (Węzeł) ---")
a2, K2, b2, f2, c2 = 0.5, 1000, 0.01, 0.01, 0.5
print(f"Parametry: a={a2}, K={K2}, b={b2}, f={f2}, c={c2}")
print(f"Sprawdzenie warunku bK > c: {f2 * K2} > {c2} -> {f2 * K2 > c2}")

X_star2 = c2 / f2
Y_star2 = (a2 / b2) * (1 - c2 / (f2 * K2))
print(f"Obliczony punkt równowagi P2: X*={X_star2:.2f}, Y*={Y_star2:.2f}")

# Warunki początkowe dla scenariusza 2
M0_2 = [X_star2 + 10, Y_star2 + 10]

# Symulacja dla scenariusza 2
solution2_M = odeint(modified_lotka_volterra, M0_2, t_vector, args=(a2, K2, b2, f2, c2))
X_sol2, Y_sol2 = solution2_M[:, 0], solution2_M[:, 1]

#animacja dla scenariusza 2
fig2, (ax2_time, ax2_phase) = plt.subplots(1, 2, figsize=(14, 6))

#wykres czasowy
ax2_time.set_xlim(0, t_vector.max())
ax2_time.set_ylim(0, max(X_sol2.max(), Y_sol2.max(), M0_2[0], M0_2[1]) * 1.1)
ax2_time.set_xlabel('Czas')
ax2_time.set_ylabel('Liczebność Populacji')
ax2_time.set_title('Scenariusz 2: Wykres Czasowy (Węzeł)')
ax2_time.grid(True)
line2_X_time, = ax2_time.plot([], [], lw=2, label='Ofiary (X)', color='blue')
line2_Y_time, = ax2_time.plot([], [], lw=2, label='Drapieżniki (Y)', color='red')
ax2_time.legend()

#wykres fazowy
ax2_phase.set_xlim(0, max(X_sol2.max(), M0_2[0]) * 1.1)
ax2_phase.set_ylim(0, max(Y_sol2.max(), M0_2[1]) * 1.1)
ax2_phase.set_xlabel('Liczebność Ofiar (X)')
ax2_phase.set_ylabel('Liczebność Drapieżników (Y)')
ax2_phase.set_title('Scenariusz 2: Portret Fazowy (Węzeł)')
ax2_phase.grid(True)
line2_phase, = ax2_phase.plot([], [], lw=2, color='purple')
point2_phase, = ax2_phase.plot([], [], 'o', color='purple', markersize=5)
ax2_phase.plot(X_star2, Y_star2, 'go', markersize=8, label='Punkt równowagi P2')
ax2_phase.legend()

plt.tight_layout()

def init2():
    line2_X_time.set_data([], [])
    line2_Y_time.set_data([], [])
    line2_phase.set_data([], [])
    point2_phase.set_data([], [])
    return line2_X_time, line2_Y_time, line2_phase, point2_phase

def animate2(i):
    line2_X_time.set_data(t_vector[:i], X_sol2[:i])
    line2_Y_time.set_data(t_vector[:i], Y_sol2[:i])
    line2_phase.set_data(X_sol2[:i], Y_sol2[:i])
    if i > 0:
      point2_phase.set_data(X_sol2[i-1:i], Y_sol2[i-1:i])
    else:
      point2_phase.set_data([], [])
    return line2_X_time, line2_Y_time, line2_phase, point2_phase

anim2 = FuncAnimation(fig2, animate2, init_func=init2, frames=len(t_vector), interval=10, blit=True)
plt.close(fig2)
display(HTML(anim2.to_jshtml()))


In [ ]:

# --- Scenariusz 3: Stabilna Koegzystencja (P2 jako stabilne ognisko – oscylacje tłumione) ---
print("\n--- Scenariusz 3: Stabilna Koegzystencja (Ognisko - oscylacje tłumione) ---")
a3, K3, b3, f3, c3 = 1.0, 1000, 0.03, 0.01, 0.5
print(f"Parametry: a={a3}, K={K3}, b={b3}, f={f3}, c={c3}")
print(f"Sprawdzenie warunku bK > c: {f3 * K3} > {c3} -> {f3 * K3 > c3}")

X_star3 = c3 / f3
Y_star3 = (a3 / b3) * (1 - c3 / (f3 * K3))
print(f"Obliczony punkt równowagi P2: X*={X_star3:.2f}, Y*={Y_star3:.2f}")

#Warunki początkowe dla scenariusza 3
M0_3 = [X_star3 + 50, Y_star3 + 50]

#Symulacja dla scenariusza 3
solution3_M = odeint(modified_lotka_volterra, M0_3, t_vector, args=(a3, K3, b3, f3, c3))
X_sol3, Y_sol3 = solution3_M[:, 0], solution3_M[:, 1]

#Animacja dla scenariusza 3
fig3, (ax3_time, ax3_phase) = plt.subplots(1, 2, figsize=(14, 6))

#Wykres czasowy
ax3_time.set_xlim(0, t_vector.max())
ax3_time.set_ylim(0, max(np.max(X_sol3), np.max(Y_sol3), M0_3[0], M0_3[1]) * 1.1) # Użycie np.max dla bezpieczeństwa
ax3_time.set_xlabel('Czas')
ax3_time.set_ylabel('Liczebność Populacji')
ax3_time.set_title('Scenariusz 3: Wykres Czasowy (Ognisko)')
ax3_time.grid(True)
line3_X_time, = ax3_time.plot([], [], lw=2, label='Ofiary (X)', color='blue')
line3_Y_time, = ax3_time.plot([], [], lw=2, label='Drapieżniki (Y)', color='red')
ax3_time.legend()

#Wykres fazowy
ax3_phase.set_xlim(0, max(np.max(X_sol3), M0_3[0]) * 1.1)
ax3_phase.set_ylim(0, max(np.max(Y_sol3), M0_3[1]) * 1.1)
ax3_phase.set_xlabel('Liczebność Ofiar (X)')
ax3_phase.set_ylabel('Liczebność Drapieżników (Y)')
ax3_phase.set_title('Scenariusz 3: Portret Fazowy (Ognisko)')
ax3_phase.grid(True)
line3_phase, = ax3_phase.plot([], [], lw=2, color='purple')
point3_phase, = ax3_phase.plot([], [], 'o', color='purple', markersize=5)
ax3_phase.plot(X_star3, Y_star3, 'go', markersize=8, label='Punkt równowagi P2')
ax3_phase.legend()

plt.tight_layout()

def init3():
    line3_X_time.set_data([], [])
    line3_Y_time.set_data([], [])
    line3_phase.set_data([], [])
    point3_phase.set_data([], [])
    return line3_X_time, line3_Y_time, line3_phase, point3_phase

def animate3(i):
    line3_X_time.set_data(t_vector[:i], X_sol3[:i])
    line3_Y_time.set_data(t_vector[:i], Y_sol3[:i])
    line3_phase.set_data(X_sol3[:i], Y_sol3[:i])
    if i > 0:
      point3_phase.set_data(X_sol3[i-1:i], Y_sol3[i-1:i])
    else:
      point3_phase.set_data([], [])
    return line3_X_time, line3_Y_time, line3_phase, point3_phase

anim3 = FuncAnimation(fig3, animate3, init_func=init3, frames=len(t_vector), interval=10, blit=True)
plt.close(fig3)
display(HTML(anim3.to_jshtml()))

In [ ]:
# --- Scenariusz 4: Stabilna Koegzystencja  ---
print("\n--- Scenariusz 3: Stabilna Koegzystencja ---")
a4, K4, b4, f4, c4 = 1.0, 100, 0.02, 0.01, 0.1
print(f"Parametry: a={a4}, K={K4}, b={b4}, f={f4}, c={c4}")
print(f"Sprawdzenie warunku bK > c: {f4 * K4} > {c4} -> {f4 * K4 > c4}")

X_star4 = c4 / f4
Y_star4 = (a4 / b4) * (1 - c4 / (f4 * K4))
print(f"Obliczony punkt równowagi P4: X*={X_star4:.2f}, Y*={Y_star4:.2f}")

#Warunki początkowe dla scenariusza 4
M0_4 = [X_star4 + 50, Y_star4 + 50]

#Symulacja dla scenariusza 4
solution4_M = odeint(modified_lotka_volterra, M0_4, t_vector, args=(a4, K4, b4, f4, c4))
X_sol4, Y_sol4 = solution4_M[:, 0], solution4_M[:, 1]

#Animacja dla scenariusza 4
fig4, (ax4_time, ax4_phase) = plt.subplots(1, 2, figsize=(14, 6))

#Wykres czasowy
ax4_time.set_xlim(0, t_vector.max())
ax4_time.set_ylim(0, max(np.max(X_sol4), np.max(Y_sol4), M0_4[0], M0_4[1]) * 1.1) # Użycie np.max dla bezpieczeństwa
ax4_time.set_xlabel('Czas')
ax4_time.set_ylabel('Liczebność Populacji')
ax4_time.set_title('Scenariusz 4: Wykres Czasowy (Ognisko)')
ax4_time.grid(True)
line4_X_time, = ax4_time.plot([], [], lw=2, label='Ofiary (X)', color='blue')
line4_Y_time, = ax4_time.plot([], [], lw=2, label='Drapieżniki (Y)', color='red')
ax4_time.legend()

#wykres fazowy
ax4_phase.set_xlim(0, max(np.max(X_sol4), M0_4[0]) * 1.1)
ax4_phase.set_ylim(0, max(np.max(Y_sol4), M0_4[1]) * 1.1)
ax4_phase.set_xlabel('Liczebność Ofiar (X)')
ax4_phase.set_ylabel('Liczebność Drapieżników (Y)')
ax4_phase.set_title('Scenariusz 4: Portret Fazowy (Ognisko)')
ax4_phase.grid(True)
line4_phase, = ax4_phase.plot([], [], lw=2, color='purple')
point4_phase, = ax4_phase.plot([], [], 'o', color='purple', markersize=5)
ax4_phase.plot(X_star4, Y_star4, 'go', markersize=8, label='Punkt równowagi P3')
ax4_phase.legend()

plt.tight_layout()

def init4():
    line4_X_time.set_data([], [])
    line4_Y_time.set_data([], [])
    line4_phase.set_data([], [])
    point4_phase.set_data([], [])
    return line4_X_time, line4_Y_time, line4_phase, point4_phase

def animate4(i):
    line4_X_time.set_data(t_vector[:i], X_sol4[:i])
    line4_Y_time.set_data(t_vector[:i], Y_sol4[:i])
    line4_phase.set_data(X_sol4[:i], Y_sol4[:i])
    if i > 0:
      point4_phase.set_data(X_sol4[i-1:i], Y_sol4[i-1:i])
    else:
      point4_phase.set_data([], [])
    return line4_X_time, line4_Y_time, line4_phase, point4_phase

anim4 = FuncAnimation(fig4, animate4, init_func=init4, frames=len(t_vector), interval=10, blit=True)
plt.close(fig4)
display(HTML(anim4.to_jshtml()))

In [ ]:
# --- Scenariusz 5: Zwiększenie współczynnika wzrostu ofiar ---
print("\n--- Scenariusz 5: Stabilna Koegzystencja (Ognisko - oscylacje tłumione) ---")
a5, K5, b5, f5, c5 = 1.5, 1000, 0.02, 0.01, 0.2
print(f"Parametry: a={a5}, K={K5}, b={b5}, f={f5}, c={c5}")
print(f"Sprawdzenie warunku bK > c: {f5 * K5} > {c5} -> {f5 * K5 > c5}")

X_star5 = c5 / f5
Y_star5 = (a5 / b5) * (1 - c5 / (f5 * K5))
print(f"Obliczony punkt równowagi P5: X*={X_star5:.2f}, Y*={Y_star5:.2f}")

#Warunki początkowe dla scenariusza 5
M0_5 = [X_star5 + 50, Y_star5 + 50]

#Symulacja dla scenariusza 5
solution5_M = odeint(modified_lotka_volterra, M0_5, t_vector, args=(a5, K5, b5, f5, c5))
X_sol5, Y_sol5 = solution5_M[:, 0], solution5_M[:, 1]

#Animacja dla scenariusza 5
fig5, (ax5_time, ax5_phase) = plt.subplots(1, 2, figsize=(14, 6))

#Wykres czasowy
ax5_time.set_xlim(0, t_vector.max())
ax5_time.set_ylim(0, max(np.max(X_sol5), np.max(Y_sol5), M0_5[0], M0_5[1]) * 1.1) # Użycie np.max dla bezpieczeństwa
ax5_time.set_xlabel('Czas')
ax5_time.set_ylabel('Liczebność Populacji')
ax5_time.set_title('Scenariusz 5: Wykres Czasowy (Ognisko)')
ax5_time.grid(True)
line5_X_time, = ax5_time.plot([], [], lw=2, label='Ofiary (X)', color='blue')
line5_Y_time, = ax5_time.plot([], [], lw=2, label='Drapieżniki (Y)', color='red')
ax5_time.legend()

#wykres fazowy
ax5_phase.set_xlim(0, max(np.max(X_sol5), M0_5[0]) * 1.1)
ax5_phase.set_ylim(0, max(np.max(Y_sol5), M0_5[1]) * 1.1)
ax5_phase.set_xlabel('Liczebność Ofiar (X)')
ax5_phase.set_ylabel('Liczebność Drapieżników (Y)')
ax5_phase.set_title('Scenariusz 5: Portret Fazowy (Ognisko)')
ax5_phase.grid(True)
line5_phase, = ax5_phase.plot([], [], lw=2, color='purple')
point5_phase, = ax5_phase.plot([], [], 'o', color='purple', markersize=5)
ax5_phase.plot(X_star5, Y_star5, 'go', markersize=8, label='Punkt równowagi P5')
ax5_phase.legend()

plt.tight_layout()

def init5():
    line5_X_time.set_data([], [])
    line5_Y_time.set_data([], [])
    line5_phase.set_data([], [])
    point5_phase.set_data([], [])
    return line5_X_time, line5_Y_time, line5_phase, point5_phase

def animate5(i):
    line5_X_time.set_data(t_vector[:i], X_sol5[:i])
    line5_Y_time.set_data(t_vector[:i], Y_sol5[:i])
    line5_phase.set_data(X_sol5[:i], Y_sol5[:i])
    if i > 0:
      point5_phase.set_data(X_sol5[i-1:i], Y_sol5[i-1:i])
    else:
      point5_phase.set_data([], [])
    return line5_X_time, line5_Y_time, line5_phase, point5_phase

anim5 = FuncAnimation(fig5, animate5, init_func=init5, frames=len(t_vector), interval=10, blit=True)
plt.close(fig5)
display(HTML(anim5.to_jshtml()))

In [ ]:
# --- Scenariusz 6: Zwiększenie współczynnika reprodukcji drapieżnika ---
print("\n--- Scenariusz 6: Stabilna Koegzystencja (Ognisko - oscylacje tłumione) ---")
a6, K6, b6, f6, c6 = 1.0, 1000, 0.02, 0.015, 0.2
print(f"Parametry: a={a6}, K={K6}, b={b6}, f={f6}, c={c6}")
print(f"Sprawdzenie warunku bK > c: {f6 * K6} > {c6} -> {f6 * K6 > c6}")

X_star6 = c6 / f6
Y_star6 = (a6 / b6) * (1 - c6 / (f6 * K6))
print(f"Obliczony punkt równowagi P5: X*={X_star6:.2f}, Y*={Y_star6:.2f}")

#Warunki początkowe dla scenariusza 6
M0_6 = [X_star6 + 50, Y_star6 + 50]

#Symulacja dla scenariusza 6
solution6_M = odeint(modified_lotka_volterra, M0_6, t_vector, args=(a6, K6, b6, f6, c6))
X_sol6, Y_sol6 = solution6_M[:, 0], solution6_M[:, 1]

#Animacja dla scenariusza 6
fig6, (ax6_time, ax6_phase) = plt.subplots(1, 2, figsize=(14, 6))


#wykres czasowy
ax6_time.set_xlim(0, t_vector.max())
ax6_time.set_ylim(0, max(np.max(X_sol6), np.max(Y_sol6), M0_6[0], M0_6[1]) * 1.1) # Użycie np.max dla bezpieczeństwa
ax6_time.set_xlabel('Czas')
ax6_time.set_ylabel('Liczebność Populacji')
ax6_time.set_title('Scenariusz 6: Wykres Czasowy (Ognisko)')
ax6_time.grid(True)
line6_X_time, = ax6_time.plot([], [], lw=2, label='Ofiary (X)', color='blue')
line6_Y_time, = ax6_time.plot([], [], lw=2, label='Drapieżniki (Y)', color='red')
ax6_time.legend()


#Wykres fazowy
ax6_phase.set_xlim(0, max(np.max(X_sol6), M0_6[0]) * 1.1)
ax6_phase.set_ylim(0, max(np.max(Y_sol6), M0_6[1]) * 1.1)
ax6_phase.set_xlabel('Liczebność Ofiar (X)')
ax6_phase.set_ylabel('Liczebność Drapieżników (Y)')
ax6_phase.set_title('Scenariusz 6: Portret Fazowy (Ognisko)')
ax6_phase.grid(True)
line6_phase, = ax6_phase.plot([], [], lw=2, color='purple')
point6_phase, = ax6_phase.plot([], [], 'o', color='purple', markersize=5)
ax6_phase.plot(X_star6, Y_star6, 'go', markersize=8, label='Punkt równowagi P6')
ax6_phase.legend()

plt.tight_layout()

def init6():
    line6_X_time.set_data([], [])
    line6_Y_time.set_data([], [])
    line6_phase.set_data([], [])
    point6_phase.set_data([], [])
    return line6_X_time, line6_Y_time, line6_phase, point6_phase

def animate6(i):
    line6_X_time.set_data(t_vector[:i], X_sol6[:i])
    line6_Y_time.set_data(t_vector[:i], Y_sol6[:i])
    line6_phase.set_data(X_sol6[:i], Y_sol6[:i])
    if i > 0:
      point6_phase.set_data(X_sol6[i-1:i], Y_sol6[i-1:i])
    else:
      point6_phase.set_data([], [])
    return line6_X_time, line6_Y_time, line6_phase, point6_phase

anim6 = FuncAnimation(fig6, animate6, init_func=init6, frames=len(t_vector), interval=10, blit=True)
plt.close(fig6)
display(HTML(anim6.to_jshtml()))